# Lab Class ML:III

## Ankit Satpute, 120825 ; Hsueh Wei, 120820; Sagar Nagaraj Simha, 120797 - (M.Sc. CS4DM) - Group 13

## Exercise 3 - Algorithm ID3 and Measuring Performance

## 3(a)

In [39]:
import collections
from collections import Counter
import pandas as pd
import math

In [40]:
mushrooms_examples = [{"Color": "red", "Size": "small", "Points": "yes", "Eatability": "toxic"},
{"Color": "brown", "Size": "small", "Points": "no", "Eatability": "edible"},
{"Color": "brown", "Size": "large", "Points": "yes", "Eatability": "edible"},
{"Color": "green", "Size": "small", "Points": "no", "Eatability": "edible"},
{"Color": "red", "Size": "large", "Points": "no", "Eatability": "edible"}]

In [41]:
mushrooms_examples

[{'Color': 'red', 'Size': 'small', 'Points': 'yes', 'Eatability': 'toxic'},
 {'Color': 'brown', 'Size': 'small', 'Points': 'no', 'Eatability': 'edible'},
 {'Color': 'brown', 'Size': 'large', 'Points': 'yes', 'Eatability': 'edible'},
 {'Color': 'green', 'Size': 'small', 'Points': 'no', 'Eatability': 'edible'},
 {'Color': 'red', 'Size': 'large', 'Points': 'no', 'Eatability': 'edible'}]

In [42]:
mushrooms = pd.DataFrame(mushrooms_examples)
mushrooms

,Color,Size,Points,Eatability
0,red,small,yes,toxic
1,brown,small,no,edible
2,brown,large,yes,edible
3,green,small,no,edible
4,red,large,no,edible


In [43]:
mushrooms.columns 

Index(['Color', 'Size', 'Points', 'Eatability'], dtype='object')

In [44]:
Attributes = {'Color', 'Size', 'Points'}
target = 'Eatability'

In [45]:
# Returns the class that is most frequent
def mostCommonClass(examples, target):
    return examples[target].mode()[0]

In [46]:
mostCommonClass(mushrooms,'Eatability')

'edible'

In [47]:
# Calculates the prior probabilities of each attribute
def priori_probab(examples, class_):    
    marg_prob = {}
    for column in examples:
        d1 = collections.Counter(examples[column])
        d2={}
        for k, v in d1.items():
            d2[k] = v/len(examples)
        marg_prob[column] = d2
    return marg_prob[class_]

In [48]:
# Conditional probability of attribute 1 given attribute 2
def cond_prob(D_examples, var1 ,var2):
    #return D_examples.groupby(var1)[var2].value_counts() / len_D_examples
    return D_examples.groupby(var1)[var2].value_counts() / D_examples.groupby(var1)[var2].count()

In [49]:
def log_2(x):
    if x != 0:
        return math.log(x,2)
    else:
        return 0

In [50]:
# Conditional Entropy H. Formula followed is as per slides/notebook on Conditional Entropy
def cond_entropy(examples, attribute, target):
    pripo = priori_probab(examples, attribute)
    condipo =  (cond_prob(examples, attribute, target)).to_dict()
    cond_ent = 0
    for i in pripo:
        h_inner =0
        for j in set(examples[target]):
            try:
                h_inner -= (condipo[(i,j)] * log_2(condipo[(i,j)]))
            except KeyError:
                continue
        cond_ent += (pripo[(i)] * (h_inner))
    return (cond_ent)

In [51]:
cond_entropy(mushrooms, 'Points', 'Eatability')

0.4

In [52]:
cond_entropy(mushrooms, 'Size', 'Eatability')

0.5509775004326937

In [53]:
# returns the attribute with the minimum entropy, which is nothing but the attribute that provides 
# maximum information since the entropy of the orginial dataset is constant
def information_gain_attribute(examples, attributes, target):
    att_ce = {} 
    for att in attributes:
        att_ce.update({att:cond_entropy(examples,att,target)})
    return min(att_ce, key=att_ce.get) # In case of same conditional entropy the function returns one of the keys

In [54]:
information_gain_attribute(mushrooms, Attributes, target)

'Color'

In [55]:
## Test cases from exercise
D_examples = [dict(a=a,b=b,c=c,d=d) for a,b,c,d in zip("1111", "1122", "1212", "1233")]
D_examples = pd.DataFrame(D_examples)
len_D_examples = len(D_examples)
print(cond_entropy(D_examples, "a", "d"))
print(cond_entropy(D_examples, "b", "d"))
print(cond_entropy(D_examples, "c", "d"))
print(cond_entropy(D_examples, "d", "d"))
print(cond_entropy(D_examples, "d", "c"))

1.5
0.5
1.0
0.0
0.5


## 3(b)

In [56]:
class ID3Node():
    # Intitialization of attribute and label to None
    # children is a dictionary which is initailized to empty
    # children will contain a dictionary with key as attribute and value as another ID3 Node as and when
    # create_edge is called
    def __init__(self):
        self.attribute = None
        self.label = None
        self.children={}
        
    #Creates an edge/connection between attribute value and
    #another ID3 node, which is a key value pair
    def create_edge(self, attribute_value, child):
        self.children[attribute_value] = child
        
    
    #classify will return the label of the final leaf node
    def classify(self, example):
        # If there are no child nodes, i.e the dictionary is empty, then return the curent node's label
        if(bool(self.children)==False):
            return self.label
        else:
            #try:
            attr_value_ = example[self.attribute]
            #except:
                #print("Exception here", self.attribute)
            #if attr_value_ as a key is in children, then go into the next child node to search for label using recursion
            if attr_value_ in self.children.keys():
                return self.children[attr_value_].classify(example)
            else:
                #if attr_value_ is not in children, then return the current label
                return self.label

In [57]:
t = ID3Node()
t1 = ID3Node()
t2 = ID3Node()
t1.label = True
print(t1.classify({}))
t2.label = 42
print(t2.classify({}))
t.attribute = "points"
t.create_edge("yes", t1)
t.create_edge("no", t2)
print(t.classify({"points": "yes"}))
print(t.classify({"points": "no"}))

True
42
True
42


## 3(c) ID3 algorithm

In [58]:
# ID3 Algorithm
def ID3(D, Attributes, Target):
    t = ID3Node()
    t.label = mostCommonClass(D,Target)
    if D[Target].nunique() == 1 or len(Attributes)== 0:
        return t
    #A_ - Attribute selected to be the splitting criterion,
    A_ = information_gain_attribute(D, Attributes, target)
    #print(A_)
    
    t.attribute = A_
    #Regroups data D based on splitting criterion A_
    grp_D = D.groupby(A_)
    # for each unique value in attribute
    for a, item in grp_D:
        #print(a, "key")
        #print(D.get_group(a), "\n\n")
        Da = grp_D.get_group(a)
        if Da.empty:
            t_ = ID3Node()
            t_.label = mostCommonClass(D, Target)
            t.create_edge(a,t_)
        else:
            t.create_edge(a, ID3(Da, Attributes.difference([A_]), Target) )
    return t

In [21]:
# Generates the tree 't' based on Target 'Eatability'
t = ID3(mushrooms, {'Color', 'Size', 'Points'}, 'Eatability')

In [22]:
# Test cases for classification
t.classify({'Color': 'red', 'Points': 'yes', 'Size': 'small'})

'toxic'

In [23]:
t.classify({'Color': 'brown', 'Points': 'yes', 'Size': 'large'})

'edible'

## 3(d) Misclassification rate

In [24]:
# Performing a count operation to calculate the fraction of wrong predictions from total examples
def misclassification_rate(tree, examples, target):
    correct = 0
    wrong = 0
    for each in examples:
        #print(each)
        value_from_tree = tree.classify(each)
        if value_from_tree == each[target]:
            correct+=1
        else:
            wrong+=1
    return wrong/ len(examples)

In [25]:
# Misclassification rate of the tree with respect to the examples it was trained on
misclassification_rate(t, mushrooms_examples, target)

0.0

## 3(e) Decision tree over the “Adult” dataset

In [26]:
col_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex',
        'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income']
adult = pd.read_csv('adult.data', names=col_names, index_col= False)
adult

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [27]:
# Dropping the selected columns as mentioned in the exercise
adult = adult.drop(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'],axis = 1)
adult

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...
32556,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States,<=50K
32557,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States,>50K
32558,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States,<=50K
32559,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States,<=50K


In [28]:
# Splitting data into train and test data set
adult_train = adult.iloc[:20000]
adult_test = adult.iloc[20000:]

In [29]:
adult_train

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...
19995,Self-emp-not-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,United-States,>50K
19996,Private,12th,Married-civ-spouse,Other-service,Husband,White,Male,Dominican-Republic,<=50K
19997,Private,7th-8th,Widowed,Other-service,Not-in-family,White,Female,?,<=50K
19998,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,>50K


In [30]:
# Resetting the index from 0 onwards
adult_test = adult_test.reset_index(drop=True)
adult_test

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
0,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States,<=50K
1,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,>50K
2,Private,Bachelors,Never-married,Exec-managerial,Not-in-family,White,Male,United-States,<=50K
3,?,Assoc-acdm,Widowed,?,Not-in-family,White,Female,United-States,<=50K
4,Private,HS-grad,Divorced,Craft-repair,Not-in-family,White,Male,United-States,<=50K
...,...,...,...,...,...,...,...,...,...
12556,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States,<=50K
12557,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States,>50K
12558,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States,<=50K
12559,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States,<=50K


In [31]:
# Converting the pandas dataframe to dictionary
adult_test_dic = adult_test.T.to_dict().values()

In [32]:
#Attributes of the dataset
attributes_adult = {'workclass', 'education', 'marital-status', 'occupation','relationship', 'race', 'sex', 'native-country'}

In [33]:
#Target class
target = 'income'

In [34]:
#Computes and returns the tree object trained on the adult train dataset
tree_adult = ID3(adult_train, attributes_adult, 'income')

In [35]:
# Misclassification rate of the adult TEST dataset using the generated tree structure
misclassification_rate(tree_adult, adult_test_dic, target)

0.18422100151261842

In [36]:
# Converting the pandas dataframe to dictionary
adult_train_dic = adult_train.T.to_dict().values()

In [37]:
# Misclassification rate of the adult TRAIN dataset using the generated tree structure
misclassification_rate(tree_adult, adult_train_dic, target)

0.12875

In [38]:
# Reference
#https://youtu.be/UdTKxGQvYdc
#https://github.com/codewrestling/DecisionTree/blob/master/Decision%20Tree.pdf
#https://www.uni-weimar.de/en/media/chairs/computer-science-department/webis/teaching/previous-semesters/ws-201819/machine-learning/
#http://www.ke.tu-darmstadt.de/lehre/archiv/ws0809/mldm/dt.pdf